In [ ]:
"""
    2015 11 30 Classe utilizzata per esplorare il dataset coop su cui fare analisi temporale
    
    1: Estrazione dal json dei dati temporali
    2: Calcolo distribuzioni
    3: Calcolo entropia annuale
    4: Outlier detection
    5: Entropia pesata su dati filtrati con 4
    6: Calcolo traiettorie di spesa
    7: Clustering traiettorie di spesa e ricerca outlier
    
"""

In [ ]:
#  1: Estrazione dal json dei dati temporali
# lettura file json e  estrazione della componenente temporale degli acquisti
# esportazione in un nuovo dataframe per calcolare le statistiche

import gzip
import json
infile = '../data/dataset_shop70_coopometro.json.gz'
fI = gzip.open(infile)

outfile='../data/dataset_shop70_coopometro_temporal_part.csv.gz'
fO = gzip.open(outfile, 'wb')
fO.write("customer_id;scontrino_id;anno;mese;giorno;ora;giorno_settimana;settimana_anno;quantita;importo\n")
for rec in fI:
    customer=json.loads(rec)
    for key in customer['data']:
        fO.write("%s;%s;%s;%s;%s;%s;%s;%s;%s;%s\n" %(customer['customer_id'],customer['data'][key]['scontrino_id'],
                                                    customer['data'][key]['anno'],
                                                    customer['data'][key]['mese_n'],
                                                     customer['data'][key]['giorno_n'],
                                                     customer['data'][key]['ora'],
                                                     customer['data'][key]['giorno_settimana_n'],
                                                     customer['data'][key]['settimana_anno'],
                                                     int(sum([v[0] for v in customer['data'][key]['basket'].values()])),
                                                     int(sum([v[1] for v in customer['data'][key]['basket'].values()])),
                                                    ))
    fO.flush()
fO.flush()
fO.close()

In [ ]:
#caricamento dataframe di cui sopra
import pandas as pd
outfile='../data/dataset_shop70_coopometro_temporal_part.csv.gz'
df=pd.read_csv(outfile, compression='gzip', sep=';')


In [ ]:
df.head()

In [ ]:
#  2: Calcolo distribuzioni
#distribuzioni scontrino
import pylab
path='./'
for col in list(df.columns)[2:-2]:
    s=pd.DataFrame(df.groupby([col])['scontrino_id'].count())
    s.plot(kind='bar')
    pylab.xticks(rotation=0)
    pylab.xlabel(col, fontsize=16)
    pylab.ylabel("#scontrini",fontsize=16)
    pylab.savefig("%s%s.png" %(path,col))
    pylab.clf()

In [ ]:
#  2: Calcolo distribuzioni
#distribuzioni quantita
import pylab
path='./'
for attr_raggr in (['quantita','importo']):
    for col in list(df.columns)[2:-2]:
        s=pd.DataFrame(df.groupby([col])[attr_raggr].sum())
        s.plot(kind='bar')
        if len(s)>10:
            locs, labels = pylab.xticks()
            pylab.xticks(locs[::5], list(s.ix[:,0][::5].index))
        pylab.xticks(rotation=0)
        pylab.xlabel(col, fontsize=16)
        pylab.ylabel("#oggetti acquistati",fontsize=16)
        pylab.savefig("%s%s_%s.png" %(path,col,attr_raggr))
        pylab.clf()

In [ ]:
#  2: Calcolo distribuzioni
#count dei momenti distinti
import pylab
path='./'
for col in list(df.columns)[3:-2]:
    df_distinct=pd.DataFrame(df.groupby(['customer_id','anno'])[col].apply(lambda x: len(x.unique()))).reset_index()
    df_distinct=pd.DataFrame(df_distinct.groupby([col])['customer_id'].count()  )
    df_distinct.plot(kind='bar')
    pylab.xticks(rotation=0)
    pylab.ylabel('#customers',fontsize=16)
    pylab.xlabel(col,fontsize=16)
    pylab.savefig("%s%s_distinct.png" %(path,col))
    pylab.clf()
pylab.close()

In [ ]:
#  2: Calcolo distribuzioni
#distribuzione quantita e import
import pylab
path='./'
for col in list(df.columns)[-2:]:
    df[col].hist(bins=100)
    pylab.yscale('log')
    pylab.ylabel('#scontrini',fontsize=16)
    pylab.xlabel(col,fontsize=16)
    pylab.savefig("%s%s.png" %(path,col))
    pylab.clf()
pylab.close()

In [ ]:
#  2: Calcolo distribuzioni
# spesa media
df['media']=df['importo']/df['quantita']

path='./'
df['media'].hist(bins=300)
pylab.yscale('log')
pylab.ylabel('#scontrini',fontsize=16)
pylab.xlabel('importo/quantita')
pylab.savefig("%simporto_diviso_quantita.png" %(path))


In [ ]:
#  2: Calcolo distribuzioni
#distribuzione delle mediane
import pylab
import datetime
path='./'
vals={'media':[],'quantita':[],'importo':[]}
users=list(df['customer_id'].unique())
i=0
for uid in users:
    i+=1
    if i%1000==0:
        print "%s: %s users" %(datetime.datetime.now(),i)
    curr_user=df[df['customer_id']==uid]
    for col in ['media','quantita','importo']:
        vals[col].append(list(curr_user.sort(col)[col])[int(len(curr_user)/2)])

for key in vals:
    pylab.hist(vals[key],100)
    pylab.ylabel('#scontrini',fontsize=16)
    pylab.xlabel('%s' %(key), fontsize=16)
    pylab.savefig("%s%s_%s.png" %(path,key,'mediana'))
    pylab.clf()
mediane=pd.DataFrame(vals).to_csv('../data/dataset_shop70_coopometro.json.mediana_acquisti') #manca id della persona

In [ ]:
#3: Calcolo entropia annuale
import datetime
df_entropy=pd.DataFrame()
users=list(df['customer_id'].unique())
cols=list(df.columns[3:-2])
i=0
print "%s: start" %(datetime.datetime.now())
for cus in users:
    curr_user=df[df['customer_id']==cus].copy()
    i+=1
    if i%1000==0:
        print "%s: performed %s users" %(datetime.datetime.now(),i)
        
    for anno in np.arange(2007,2015):
        dfs=[]
        for col in cols:
            dfs.append(pd.DataFrame.from_dict({cus:get_entropy(curr_user[curr_user['anno']==anno],col)},orient='index'))
        dfs.append(pd.DataFrame.from_dict({cus:{'anno':anno}},orient='index')) #join dei df dello stesso anno 
        df_entropy=df_entropy.append(pd.concat(dfs,axis=1)) #aggiunta del df dell'anno corrente agli altri anni
df_entropy.to_csv('../data/dataset_shop70_coopometro.json.entropy')

In [ ]:
def get_entropy (curr_user,col):
    """
        Calcola l'entropia per il l'utente corrente per la colonna col
        :param curr_user dataframe:dataframe dell'utente
        :col text: colonna su cui calcolare l'entropia
        :return d dict: dizionario {col:entropy}
    """
    import numpy as np
    s=pd.DataFrame(curr_user.groupby([col])['scontrino_id'].count())
    s['tot']=[sum(s['scontrino_id'])]*len(s)
    if col=='giorno':
        num_classi=31
    elif col=='mese':
        num_classi=12
    elif col=='ora':
        num_classi=14
    elif col=='giorno_settimana':
        num_classi=7
    elif col=='settimana_anno':
        num_classi=53
    return {col:(-1*sum(s['scontrino_id']/s['tot']*np.log(s['scontrino_id']/s['tot']))/np.log(num_classi))}


In [ ]:
#statistica sulla entropia
import pylab
path='./'
#tolgo i casi in cui ho zero in tutte le entropie perchè non avevo dati
df_entropy=df_entropy[(df_entropy['mese']<>0) & (df_entropy['giorno']<>0)& (df_entropy['ora']<>0)& (df_entropy['giorno_settimana']<>0) & (df_entropy['settimana_anno']<>0) ]
for col in list(df_entropy.columns)[0:-1]:
    df_entropy[col].hist()
    pylab.xticks(rotation=0)
    pylab.title(col,fontsize=20)
    pylab.xlabel("entropy", fontsize=16)
    pylab.ylabel("#users",fontsize=16)
    pylab.savefig("%s%s_entropy.png" %(path,col))
    pylab.clf()

In [ ]:
#4 Eliminazione outlier usando MAD e IQR
import gzip
import datetime

#nuovi file
f_iqr=gzip.open('../data/dataset_shop70_coopometro_temporal_part_filtered_iqr.csv.gz','w')
f_mad=gzip.open('../data/dataset_shop70_coopometro_temporal_part_filtered_mad.csv.gz','w')
f_iqr.write(';'.join(list(df.columns))+"\n")
f_mad.write(';'.join(list(df.columns))+"\n")

#file con statistiche
f_filter_stats=open('../data/dataset_shop70_coopometro_temporal_part_filtered_stats.csv','w')
f_filter_stats.write("customer_id;filterd_iqr;filtered_mad\n")

i=0
for uid in users:
    i+=1
    if i%1000==0:
        print "%s: users %s" %(datetime.datetime.now(),i)
    curr_user=df[df['customer_id']==uid]
    curr_user_mad=curr_user.copy()
    df_stat=curr_user[['customer_id','quantita','importo']].describe()
    init_cols=list(curr_user.columns)
    init_rows=len(curr_user)
    
    #aggiungiamo le colonne iqr e mad e i quartili
    for col in ['quantita','importo']:
        curr_user['IQR_%s'%(col)]=[df_stat[col].ix['75%']-df_stat[col].ix['25%']]*df_stat.ix['count'][0]
        curr_user['Q1_%s'%(col)]=[df_stat[col].loc['25%']]*df_stat.ix['count'][0]
        curr_user['Q3_%s'%(col)]=[df_stat[col].loc['75%']]*df_stat.ix['count'][0]

    #filtro outlier con     #IQR
    for col in ['quantita','importo']:
        curr_user=curr_user[curr_user['Q1_%s' %(col)]-1.5*curr_user['IQR_%s' %(col)]<=curr_user[col]]
        curr_user=curr_user[curr_user[col]<=curr_user['Q3_%s'%(col)]+1.5*curr_user['IQR_%s'%(col)]]

    #salvare risultato del filtro di iqr
    curr_user[init_cols].to_csv(f_iqr,header=False,sep=';',index=False)
    nrows_iqr=init_rows-len(curr_user)

    #MAD
    curr_user=curr_user_mad
    for col in ['quantita','importo']:
        res=sorted(abs(curr_user[col]-df_stat[col].loc['50%']))
        mad=res[int(len(res)/2)]
        curr_user=curr_user[curr_user[col]<=(df_stat[col].ix['50%']+3*mad*1.483)]
        curr_user=curr_user[curr_user[col]>=(df_stat[col].ix['50%']-3*mad*1.483)]

    #salvare risultato del filtro di mad
    curr_user.to_csv(f_mad,header=False,sep=';',index=False)

    #salvo risulato delle statistiche di eliminaizone
    f_filter_stats.write('%s;%s;%s\n' %(uid,nrows_iqr,(init_rows-len(curr_user))))
    
    f_iqr.flush()
    f_mad.flush()
    f_filter_stats.flush()

#chiudo output stream    
f_iqr.flush()
f_iqr.close()
f_mad.flush()
f_mad.close()
f_filter_stats.flush()
f_filter_stats.close()

In [ ]:
#statistica sul numero di dati cancellati
filter_file_stats='../data/dataset_shop70_coopometro_temporal_part_filtered_stats.csv'
df_fs=pd.read_csv(filter_file_stats, sep=';')
count_scontrini_utente=pd.DataFrame(df.groupby(['customer_id'])['scontrino_id'].count()) #num righe per utente
df_fs.index=df_fs['customer_id']
del(df_fs['customer_id'])
df_fs=pd.concat([df_fs,count_scontrini_utente],axis='inner') #non avevo salvato il totale righe per utente

In [ ]:
df_fs['perc_iqr']=df_fs['filterd_iqr']/df_fs['scontrino_id']
df_fs['perc_mad']=df_fs['filtered_mad']/df_fs['scontrino_id']
df_fs['perc_iqr'].hist(bins=np.arange(0,0.4,.05),alpha=0.50, color='r',label='IQR')
df_fs['perc_mad'].hist(bins=np.arange(0,0.4,.05),alpha=0.50, color='b',label='MAD')
pylab.xlabel('percentuale dati filtrati', fontsize=16)
pylab.ylabel('#customer_id', fontsize=16)
pylab.legend()
pylab.savefig('../experiments/20151130_analisi_esplorativa/distribuzione_dati_filtrati.png')

In [1]:
#stat sul dato iqr ottenuto filtrando con il codice due blocchi sopra
import pandas as pd
input_file='../data/dataset_shop70_coopometro_temporal_part_filtered_iqr.csv.gz'
df_iqr=pd.read_csv(input_file, compression='gzip',sep=';')

In [12]:
#definizione del processo parallelo per calcolare entropia pesata sui dati filtrati
from IPython.parallel import Client

def parallel_plot(view, users,col2):
    import numpy as np
    results=[]
    for uids,i in zip(users,np.arange(0,len(users))):
        results.append(view.apply(plot, uids,i,col2))
    
    return [r.result for r in results]

def plot(users,i,col2):
    
    def get_wheighted_entropy (curr_user,col,col2):
        """
            Calcola l'entropia per il l'utente corrente per la colonna col
            :param curr_user dataframe:dataframe dell'utente
            :param col text: colonna su cui calcolare l'entropia
            :param col2 text: colonna per cui sommare
            :return d dict: dizionario {col:entropy}
        """
        import numpy as np
        import pandas as pd
        s=pd.DataFrame(curr_user.groupby([col])[col2].sum())
        s['tot']=[sum(s[col2])]*len(s)
        num_classi=0
        if col=='giorno':
            num_classi=31
        elif col=='mese':
            num_classi=12
        elif col=='ora':
            num_classi=14
        elif col=='giorno_settimana':
            num_classi=7
        elif col=='settimana_anno':
            num_classi=53
        return {col:(-1*sum(s[col2]/s['tot']*np.log(s[col2]/s['tot']))/np.log(num_classi))}

    
    import pandas as pd
    import datetime
    import numpy as np
    
    input_file='../data/dataset_shop70_coopometro_temporal_part_filtered_iqr.csv.gz'
    df=pd.read_csv(input_file, compression='gzip',sep=';')
    f_iqr=open('../data/dataset_shop70_coopometro_temporal_part_filtered_iqr_wheighted_entropy_%s_%s.csv' %(col2,i),'a')

    
    
    cols=list(df.columns[3:-3]) #attenzione al numero di colonne che si hanno nel df, non ciclare sulle misure importo media e quantita
    i=0
    print "%s: start" %(datetime.datetime.now())
    for cus in users:
        df_entropy=pd.DataFrame()
        curr_user=df[df['customer_id']==cus].copy()
        i+=1
        if i%1000==0:
            print "%s: performed %s users" %(datetime.datetime.now(),i)

        for anno in np.arange(2007,2015):
            dfs=[]
            for col in cols:
                dfs.append(pd.DataFrame.from_dict({cus:get_wheighted_entropy(curr_user[curr_user['anno']==anno],col,col2)},orient='index'))
            dfs.append(pd.DataFrame.from_dict({cus:{'anno':anno}},orient='index')) #join dei df dello stesso anno 
            df_entropy=df_entropy.append(pd.concat(dfs,axis=1)) #aggiunta del df dell'anno corrente agli altri anni
    
        #salvo il dataframe su file
        df_entropy=df_entropy.reset_index()
        df_entropy.to_csv(f_iqr, sep=';',header=False,index=False)
    
    f_iqr.flush()
    f_iqr.close()

    return i

    
#creazione di tanti file di n elementi max
def chunks(l, n):
    n = max(1, n)
    return [l[i:i + n] for i in range(0, len(l), n)]

In [13]:
#5 Entropia su pesata sulla quantita
#run del processo
client = Client()
view = client.load_balanced_view()
users=df_iqr['customer_id'].unique()
users_=chunks(users,len(users)/3+1)
results = parallel_plot(view, users_,'importo') #una volta per quantita una per importo

In [51]:
#5 Entropia su pesata sulla quantita - distribuzioni
import pandas
for col1 in ['quantita','importo']:
    infile='../data/dataset_shop70_coopometro_temporal_part_filtered_iqr.w_entropia_%s.txt' %(col1)
    df_iqr_w_entropy=pd.read_csv(infile,sep=';',decimal='.')
    df_iqr_w_entropy=df_iqr_w_entropy.replace(-0.0,0)
    df_iqr_w_entropy=df_iqr_w_entropy.replace(-float('infinity'),0)
    #statistica sulla entropia
    import pylab
    path='./'
    #tolgo i casi in cui ho zero in tutte le entropie perchè non avevo dati
    df_iqr_w_entropy=df_iqr_w_entropy[(df_iqr_w_entropy['mese']<>0) & (df_iqr_w_entropy['giorno']<>0)& (df_iqr_w_entropy['ora']<>0)& (df_iqr_w_entropy['giorno_settimana']<>0) & (df_iqr_w_entropy['settimana_anno']<>0) ]
    for col in list(df_iqr_w_entropy.columns)[1:-1]:
        df_iqr_w_entropy[col].hist()
        pylab.xticks(rotation=0)
        pylab.title(col,fontsize=20)
        pylab.xlabel("entropy", fontsize=16)
        pylab.ylabel("#users",fontsize=16)
        pylab.savefig("%s%s_entropy_w_%s.png" %(path,col,col1))
        pylab.clf()

In [36]:
#6 disegno traiettorie
# sql example http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html
import numpy as np
df_iqr_spesa_mensile=df_iqr.groupby(['customer_id','anno']).agg({'quantita': [np.size, np.mean],'importo': [np.size, np.mean]})

import pylab
dt=pd.DataFrame(df_iqr_spesa_mensile['importo']['mean']).reset_index()
dt=pd.DataFrame(pd.pivot_table(dt,values='mean',index='customer_id',columns=['anno'])).replace('NaN',0)
dt=dt.head(5000).T
dt.plot(kind='line',alpha=0.01,color='b')
pylab.legend(['traiettorie'])
pylab.xlabel('anno',fontsize=16)
pylab.ylabel('mediana acquisti annuali', fontsize=16)
pylab.title('')
pylab.ylim(0,200)
pylab.savefig('../experiments/20151130_analisi_esplorativa/traiettorie.png')

In [ ]:
#7 Clustering traiettorie
import numpy as np
#calcolo della media della spesa per mese
df_iqr_spesa_mensile=df_iqr.groupby(['customer_id','anno']).agg({'quantita': [np.size, np.mean],'importo': [np.size, np.mean]})

#pivot table id:<anni>
import pylab
dt=pd.DataFrame(df_iqr_spesa_mensile['importo']['mean']).reset_index()
dt=pd.DataFrame(pd.pivot_table(dt,values='mean',index='customer_id',columns=['anno'])).replace('NaN',0)

In [7]:
#7 clustering di traiettorie annuali
from sklearn.cluster import KMeans
dataset=dt
mat = dataset.as_matrix()
km = KMeans(n_clusters=10)
km.fit(mat)
labels = km.labels_
# Format results as a DataFrame
results = pd.DataFrame([dataset.index,labels]).T
results.columns=['customer_id','id_cluster']
results.index=results.customer_id
del(results['customer_id'])
dataset=pd.concat([dataset,results],axis='inner')

In [14]:
#disegno del cluster
colors=[ '#1f78b4', '#b2df8a', '#fb9a99', '#e31a1c', '#33a02c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#a6cee3']
centers=pd.DataFrame(km.cluster_centers_)
centers.columns=np.arange(2007,2015)
centers.T.plot(kind='line', colors=colors,lw=3)
pylab.xticks(np.arange(2007,2015))
pylab.ylabel('media mensile', fontsize=16)
pylab.show()

#salvo il risultato del cluster assieme alle figure
dataset.groupby('id_cluster').count().to_csv('../experiments/20151130_analisi_esplorativa/traiettorie.csv')